In [1]:
cd langdata

/projects/metis2/anubha/research/Multilingual-Fig-QA/langdata


In [2]:
#Reading Dataset

In [3]:
ls

en_dev.csv    hi.csv  jv.csv  su.csv  yo.csv
en_train.csv  id.csv  kn.csv  sw.csv


In [4]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(42)

### Experiment 6.2.3

In [5]:
import pandas as pd

In [6]:
# load english data
df_original = pd.read_csv("en_train.csv")
df_original.head()

,startphrase,ending1,ending2,labels
0,Her word had the strength of titanium.,Her promises can be believed.,Her promises cannot be trusted.,0
1,Her word had the strength of a wine glass.,Her promises can be believed.,Her promises cannot be trusted.,1
2,His kisses have the passion of lovers meeting ...,His kisses are demonstrative and intense.,His kiss is unemotional.,0
3,His kisses have the passion of a couple in a l...,His kisses are demonstrative and intense.,His kiss is unemotional.,1
4,This winter is as cold as my mother-in-law tow...,It's very cold,It's pretty warm,0


In [7]:
#load other data
df = pd.read_csv("yo.csv")
df = df.reindex(df_original.columns, axis=1)
df = df.drop_duplicates()
df.head()

,startphrase,ending1,ending2,labels
0,Òkèlè ẹ̀bà mẹ́ẹ̀ẹ́dọ́gbọ̀n ni ọmọ náà jẹ...,Ọmọ náà jẹ oúnjẹ púpọ̀,Ọmọ náà kò jẹ oúnjẹ púpọ̀,0
1,Òkèlè ẹ̀bà kan ni ọmọ náà jẹ lánàá,Ọmọ náà jẹ oúnjẹ púpọ̀,Ọmọ náà kò jẹ oúnjẹ púpọ̀,1
2,Ó burú bí i àjẹ́ apọmọjẹ,Ó máa ń rẹ́rìn-ín,Ó níkà gan,1
3,Ó burú bí i ọlọ́kàn rere,Ó máa ń rẹ́rìn-ín,Ó níkà gan,0
4,Òrùn náà ń fẹyín síta bí i ọmọ jòjòló,Ilẹ̀ mọ́ kedere,Òkùnkún bolẹ̀ kan.,0


In [8]:
values = df['ending1'].value_counts(dropna=False).keys().tolist()
counts = df['ending1'].value_counts(dropna=False).tolist()
value_dict = dict(zip(values, counts))

In [9]:
value_dict

{'Ọmọ náà jẹ oúnjẹ púpọ̀': 2,
 'Táyé ń lérí ohun tí kò le ṣe ní': 2,
 'Kí ló dé tí o dúró láìṣe nǹkankan?': 2,
 'Wọ́n fìyà jẹ Adéyẹmí lójúde Ọba': 2,
 'Bàbá Àlàbí kú fúra rẹ̀': 2,
 'Ẹ má bínú': 2,
 'Oríre rẹ̀ ti fẹ́ parí': 2,
 'Ẹ jẹ́ kí á pa àṣírí ọ̀rọ̀ náà mọ́': 2,
 'Ó ti rí wàhálà': 2,
 'Délé sálọ nígbà tí ó gbúròó àwọn Ọlọ́pàá': 2,
 'Kẹ́hìndé lagídí lórí ọ̀rọ̀ náà': 2,
 'Ó ti run gbogbo dúkìá àwọn òbí rẹ̀': 2,
 'Ó sálọ': 2,
 'Adéyẹmí ṣagídí gidigidi': 2,
 'Táíwò ti lóyún': 2,
 'Ọ̀gá fìyà jẹ ọmọ iṣẹ́ rẹ': 2,
 'Gbogbo yín ẹ lọ fi ara balẹ̀': 2,
 'Àyìnlá yan Àjàlá jẹ': 2,
 'Ènìyàn tí ó máa ń ṣe eléyìí ṣe tọ̀hun ni Adéọlá': 2,
 'Kábíyèsí fi ìyà jẹ Àlàbí': 2,
 'Olúwọlé kọ̀jálẹ̀': 2,
 'Jẹ́ kí ohun burúkú ó padà wá ṣẹlẹ̀ síni': 2,
 'Olófòófó wà ní tòsí': 2,
 'Ó pàpà fara balẹ̀': 2,
 'Àgbàlagbà tó

In [10]:
#Add id to the loaded data
count =0
c=[]
for i in range(int(df.shape[0]/2)):
    c.append(count)
    c.append(count)
    count+=1
df["id"] = c  

In [11]:
df.head()

,startphrase,ending1,ending2,labels,id
0,Òkèlè ẹ̀bà mẹ́ẹ̀ẹ́dọ́gbọ̀n ni ọmọ náà jẹ...,Ọmọ náà jẹ oúnjẹ púpọ̀,Ọmọ náà kò jẹ oúnjẹ púpọ̀,0,0
1,Òkèlè ẹ̀bà kan ni ọmọ náà jẹ lánàá,Ọmọ náà jẹ oúnjẹ púpọ̀,Ọmọ náà kò jẹ oúnjẹ púpọ̀,1,0
2,Ó burú bí i àjẹ́ apọmọjẹ,Ó máa ń rẹ́rìn-ín,Ó níkà gan,1,1
3,Ó burú bí i ọlọ́kàn rere,Ó máa ń rẹ́rìn-ín,Ó níkà gan,0,1
4,Òrùn náà ń fẹyín síta bí i ọmọ jòjòló,Ilẹ̀ mọ́ kedere,Òkùnkún bolẹ̀ kan.,0,2


In [286]:
# #df_original = english data
# #df = language data
# #k = number of datapoints to be added
# #pair = in pair / not

# def adding_datapoints(df_original, df, k, pair):
#     df_final = pd.DataFrame(columns=df_original.columns)
#     if pair:
#         for i in range(k):
#             p = df.sample(n=1, random_state=42)
#         df_pair = df.sample(n=k, random_state=42)
#         print("number in df_pair", df_pair.shape)
#         for index, r in df_pair.iterrows():
#             r_id = r['id']
#             w = df.loc[df['id'] == r_id]
#             while(w.shape[0] == 0):
#                 p = df.sample(n=1, random_state=42)
#             w.sort_values(by=['labels'], inplace=True)
#             df=df.drop(w.index)
#             w= w.drop(['id'], axis=1)
#             if w.shape[0] < 2:
#                 print("w", r_id)
#             df_final = df_final.append(w)
#     else:
#         df_pair = df.sample(n=k, random_state=42)
#         df=df.drop(df_pair.index)
#         w= df_pair.drop(['id'], axis=1)
#         df_final = df_final.append(w)

# #     df_total = pd.concat([df_original,df_final])
#     df_total = df_final
#     df_test = df.drop(['id'], axis=1)
#     return df_total, df_test
# # 
    

In [13]:
#df_original = english data
#df = language data
#k = number of datapoints to be added
#pair = in pair / not

def adding_datapoints(df_original, df, k, pair):
    df_final = pd.DataFrame(columns=df_original.columns)
    if pair:
        for i in range(k):
            df_pair = df.sample(n=1, random_state=42)
#         df_pair = df.sample(n=k, random_state=42)
#         print("number in df_pair", df_pair.shape)
            for index, r in df_pair.iterrows():
                r_id = r['id']
                w = df.loc[df['id'] == r_id]
                w.sort_values(by=['labels'], inplace=True)
                df=df.drop(w.index)
                w= w.drop(['id'], axis=1)
                df_final = df_final.append(w)
    else:
        for i in range(k):
            df_pair = df.sample(n=1, random_state=42)
#         df_pair = df.sample(n=k, random_state=42)
            df=df.drop(df_pair.index)
            w= df_pair.drop(['id'], axis=1)
            df_final = df_final.append(w)

#     df_total = pd.concat([df_original,df_final])
    df_total = df_final
    df_test = df.drop(['id'], axis=1)
    return df_total, df_test

    

In [75]:
num_samples = 30
lang = "yo"

In [76]:
# #final dataframe, rows added in the end
# df_total = adding_datapoints(df_original, df, 10, True)
# df_total.tail()

#final dataframe, rows added in the end
df_total, df_test = adding_datapoints(df_original, df, num_samples, False)
print(df_total.shape)
df_total

(30, 4)


/tmp/ipykernel_27693/390302092.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(w)
/tmp/ipykernel_27693/390302092.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(w)
/tmp/ipykernel_27693/390302092.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(w)
/tmp/ipykernel_27693/390302092.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(w)
/tmp/ipykernel_27693/390302092.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_f

,startphrase,ending1,ending2,labels
468,Àyìnlá fá Àjàlá lórí,Àyìnlá yan Àjàlá jẹ,Àyìnlá ṣe òtítọ́ sí Àjàlá,1
469,Àyì̀nlá dá Àjàlá lọ́lá,Àyìnlá yan Àjàlá jẹ,Àyìnlá ṣe òtítọ́ sí Àjàlá,0
39,"Àkókò ń fà bí i ìgbín, a ò sí lè du...",Kò sí àkókò,Ọ̀pọ̀ àkòkò ló wà,1
516,Àrèó ti wẹ̀ yán kànhìnkàhìn,Àrèó kò ní ẹ̀sùn kankan lọ́rùn,Àrèó ní èsùn púpọ̀ lọ́rùn,0
327,Ìmọ̀ Bọ́lájí kò tí ì kún nípa iṣẹ́ igb...,Bọ́lájí mọ iṣẹ́ igbá fínfín dáadáa,Ìmọ̀ Bọ́lájí kò ì tó nípa igbá fínfín,1
328,Omi gàrí tí mo mu lánàá kan bí i omi ògì,Omi gàrí náà kan gan,Omi gàrí náà ta lẹ́nu,0
329,Omi gàrí tí mo mu lánàá korò bí i àgbo,Omi gàrí náà kan gan,Omi gàrí náà ta lẹ́nu,1
340,Ẹ wo bí Jíbíkẹ̀ẹ́ ṣe ń ṣe ìbàdí rẹ̀ fú...,Jíbíkẹ̀ẹ́ mọ bí wọ́n ṣé ń yí ìbàdí fu...,Ìbàdí Jíbíkẹ̀é ti gan,0
341,Ìranù ni Jíbíkẹ̀ẹ́ ṣe pẹ̀lú àwọn ọkùnrin,Jíbíkẹ̀ẹ́ mọ bí wọ́n ṣé ń yí ìbàdí fu...,Ìbàdí Jíbíkẹ̀é ti gan,1
260,"Lẹ́yìn iṣẹ́ ìjọba ti Àlàkẹ́ ń ṣe, ó tún...",Àlàkẹ́ ṣiṣẹ́ mìíràn lẹ́yìn iṣẹ́ ìjọba,Àlàkẹ́ ṣiṣẹ́ tó pọ̀ lánàá,0


In [77]:
print(df.shape)
print(df_total.shape)
print(df_test.shape)

(730, 5)
(30, 4)
(700, 4)


In [78]:
#loaction : /projects/metis2/anubha/research/Multilingual-Fig-QA/data/addition_data_winogrand -> for paired data addition
#loaction : /projects/metis2/anubha/research/Multilingual-Fig-QA/data/addition_data -> for single data addition
winogrand = r'/projects/metis2/anubha/research/Multilingual-Fig-QA/data/sample_addition_data_winogrand/'

no_winogrand = r'/projects/metis2/anubha/research/Multilingual-Fig-QA/data/sample_addition_data/'



folder_name = lang+ "/"
name = lang + "_" + str(num_samples)
df_total.to_csv(no_winogrand+'train/' +folder_name+ name+".csv", index = False)
df_test.to_csv(no_winogrand+'test/'+folder_name+ name+ ".csv", index = False)

### Removing Rows from testing data


### Experiment 6.2.2

In [53]:
import pandas as pd
# load english data
df_original = pd.read_csv("en_train.csv")
df_original.head()

,startphrase,ending1,ending2,labels
0,Her word had the strength of titanium.,Her promises can be believed.,Her promises cannot be trusted.,0
1,Her word had the strength of a wine glass.,Her promises can be believed.,Her promises cannot be trusted.,1
2,His kisses have the passion of lovers meeting ...,His kisses are demonstrative and intense.,His kiss is unemotional.,0
3,His kisses have the passion of a couple in a l...,His kisses are demonstrative and intense.,His kiss is unemotional.,1
4,This winter is as cold as my mother-in-law tow...,It's very cold,It's pretty warm,0


In [54]:
#load other data
df = pd.read_csv("kn.csv")
df = df.reindex(df_original.columns, axis=1)
df.head()

,startphrase,ending1,ending2,labels
0,ಆ ಸ್ಥಳವು ಆಸ್ಪತ್ರೆ ರೀತಿ ಶುಚಿಯಾಗಿದೆ.,ಆ ಸ್ಥಳವು ಶುಚಿಯಾಗಿದೆ,ಆ ಸ್ಥಳವು ಶುಚಿಯಾಗಿಲ್ಲ,0
1,ಆ ಸ್ಥಳವು ಮೀನಿನ ಮಾರುಕಟ್ಟೆ ರೀತಿ ಶುಚಿಯಾಗಿದೆ.,ಆ ಸ್ಥಳವು ಶುಚಿಯಾಗಿದೆ,ಆ ಸ್ಥಳವು ಶುಚಿಯಾಗಿಲ್ಲ,1
2,ಆ ಪ್ರದೇಶ ಯುದ್ಧ ಭೂಮಿಯಷ್ಟು ಹಿಂಸೆಯಿಂದ ಕೂಡಿದೆ.,ಆ ಪ್ರದೇಶ ಹಿಂಸೆಯಿಂದ ಕೂಡಿದೆ,ಆ ಪ್ರದೇಶ ಹಿಂಸೆಯಿಂದ ಕೂಡಿಲ್ಲ,0
3,ಆ ಪ್ರದೇಶ ದೇವಸ್ಥಾನದಷ್ಟು ಹಿಂಸೆಯಿಂದ ಕೂಡಿದೆ.,ಆ ಪ್ರದೇಶ ಹಿಂಸೆಯಿಂದ ಕೂಡಿದೆ,ಆ ಪ್ರದೇಶ ಹಿಂಸೆಯಿಂದ ಕೂಡಿಲ್ಲ,1
4,ಅವರ ಬಾಂಧವ್ಯ ಸ್ನೇಹಿತರ ಹಾಗೆ ಗಟ್ಟಿಯಿದೆ.,ಅವರ ಬಾಂಧವ್ಯ ಗಟ್ಟಿ ಇದೆ,ಅವರ ಬಾಂಧವ್ಯ ಗಟ್ಟಿ ಇಲ್ಲ,0


In [55]:
 df_total = pd.concat([df_original,df])

In [56]:
df_total.head()

,startphrase,ending1,ending2,labels
0,Her word had the strength of titanium.,Her promises can be believed.,Her promises cannot be trusted.,0
1,Her word had the strength of a wine glass.,Her promises can be believed.,Her promises cannot be trusted.,1
2,His kisses have the passion of lovers meeting ...,His kisses are demonstrative and intense.,His kiss is unemotional.,0
3,His kisses have the passion of a couple in a l...,His kisses are demonstrative and intense.,His kiss is unemotional.,1
4,This winter is as cold as my mother-in-law tow...,It's very cold,It's pretty warm,0


In [57]:
print(df_original.shape)
print(df.shape)
print(df_total.shape)

(1458, 4)
(1198, 4)
(2656, 4)


In [58]:
file = r'/projects/metis2/anubha/research/Multilingual-Fig-QA/data/exp6.2.2/'
name = "eng_kn"
df_total.to_csv(file+name+ ".csv", index = False)